In [1]:
username = 'meganorm-mznasrabadi'

In [2]:
import os

package_path = f'/home/{username}/MEGaNorm/'
os.chdir(package_path)

from utils.parallel import submit_jobs, check_jobs_status, collect_results
from datasets.camcan import load_camcan_data
from utils.nm import hbr_data_split

import pcntoolkit as ptk

In [3]:
### Paths and configs

mainParallel_path = os.path.join(package_path, 'src', 'mainParallel.py')

data_path = '/project/meganorm/Data/camcan/CamCAN/cc700/meg/pipeline/release005/BIDSsep/derivatives_rest/aa/AA_movecomp_transdef/aamod_meg_maxfilt_00003/'
target_dir = f'/home/{username}/Data/CamCAN/Features/'

log_path = f'/home/{username}/temp/log/'
temp_path = f'/home/{username}/temp/tmp/'

processing_dir = f'/home/{username}/temp/tmp/'

job_configs = {'log_path':log_path, 'module':'mne', 'time':'1:00:00', 'memory':'20GB', 
                'partition':'normal', 'core':1, 'node':1, 'batch_file_name':'batch_job'}

if not os.path.isdir(log_path):
    os.makedirs(log_path)

In [4]:
### Parallel feature extraction

subjects = os.listdir(data_path)
 
# Running Jobs
start_time = submit_jobs(mainParallel_path, target_dir, data_path, subjects, 
                temp_path, job_configs=job_configs)
# Checking jobs
failed_job_names= check_jobs_status(username, start_time)

while len(failed_job_names)>0:
    # Re-running Jobs
    start_time = submit_jobs(mainParallel_path, target_dir, data_path, failed_job_names, 
                temp_path, job_configs=job_configs)
    # Checking jobs
    failed_job_names= check_jobs_status(username, start_time)

collect_results(target_dir, temp_path, file_name='camcan_features')

Submitted batch job 12179736
Submitted batch job 12179737
Submitted batch job 12179738
Submitted batch job 12179739
Submitted batch job 12179740
Submitted batch job 12179741
Submitted batch job 12179742
Submitted batch job 12179743
Submitted batch job 12179744
Submitted batch job 12179745
Submitted batch job 12179746
Submitted batch job 12179747
Submitted batch job 12179748
Submitted batch job 12179749
Submitted batch job 12179750
Submitted batch job 12179751
Submitted batch job 12179752
Submitted batch job 12179753
Submitted batch job 12179754
Submitted batch job 12179755
Submitted batch job 12179756
Submitted batch job 12179757
Submitted batch job 12179758
Submitted batch job 12179759
Submitted batch job 12179760
Submitted batch job 12179761
Submitted batch job 12179762
Submitted batch job 12179763
Submitted batch job 12179764
Submitted batch job 12179765
Submitted batch job 12179766
Submitted batch job 12179767
Submitted batch job 12179768
Submitted batch job 12179769
Submitted batc

In [9]:
### Data preparation for Normative Modeling

processing_dir = f'/home/{username}/temp/tmp/'
camcan_cov_path = '/project/meganorm/Data/camcan/CamCAN/cc700/participants.tsv'
camcan_feature_path = f'/home/{username}/Data/CamCAN/Features/camcan_features.csv'

camcan_data = load_camcan_data(camcan_feature_path, camcan_cov_path)
hbr_data_split(camcan_data, processing_dir, drop_nans=True, batch_effects=['gender','site'])

              age  gender  site
sub-CC110033   24       0     0
sub-CC110037   18       0     0
sub-CC110045   24       1     0
sub-CC110056   22       1     0
sub-CC110062   20       0     0
...           ...     ...   ...
sub-CC722542   79       0     0
sub-CC722651   79       1     0
sub-CC722891   84       1     0
sub-CC723197   80       1     0
sub-CC723395   86       1     0

[653 rows x 3 columns]
              age  gender  site  width_dominant_peak_Beta_mag  \
sub-CC110033   24       0     0                      7.317979   
sub-CC110037   18       0     0                      4.380368   
sub-CC110045   24       1     0                      3.906643   
sub-CC110056   22       1     0                      6.355760   
sub-CC110069   28       1     0                      6.740317   
...           ...     ...   ...                           ...   
sub-CC722542   79       0     0                      8.279459   
sub-CC722651   79       1     0                     10.103632   
sub-CC7

In [1]:
### Setting up NM configs

python_path = '/project/meganorm/Software/Miniconda3/envs/mne/bin/python' 

hbr_configs = {'hetero_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                   'random_slope_mu':'True', 'linear_epsilon':'True', 'linear_delta':'True'},
            }

inscaler='minmax' 
outscaler='minmax' 
batch_size = 1
outputsuffix = '_estimate'

respfile = processing_dir + 'y_train.pkl'
covfile = processing_dir + 'x_train.pkl'

testrespfile_path = processing_dir + 'y_test.pkl'
testcovfile_path = processing_dir + 'x_test.pkl'

trbefile = processing_dir + 'b_train.pkl'
tsbefile = processing_dir + 'b_test.pkl'

memory = '2gb'
duration = '10:00:00'
method = 'hetero_SHASH_bspline'
cluster_spec = 'slurm'

NameError: name 'processing_dir' is not defined

In [19]:
### Running NM

ptk.normative_parallel.execute_nm(processing_dir, python_path,
               'Estimate_', covfile, respfile, batch_size, memory, duration, alg='hbr', 
               log_path=log_path, binary=True, testcovfile_path=testcovfile_path, 
               testrespfile_path=testrespfile_path,trbefile=trbefile, tsbefile=tsbefile, 
               model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
               linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
               linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
               savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix=outputsuffix, 
               interactive='auto', cluster_spec=cluster_spec)

Total Jobs:42, Pending:0, Running:0, Completed:0, Failed:0


KeyboardInterrupt: 